# Predicting USG %: An AutoRegressive Model

In [1]:
#Import Dependencies
import pandas as pd
import statsmodels.api as sm
import pickle
import numpy as np

In [2]:
#Import Data
usg = pd.read_csv('USG.csv')
raw = pd.read_csv('../Main/1998_2019_raw.csv')

The values in this dataset are proportions of team shots taken by an individual player. Columns 2 - 11 are the player's USG % 1 game ago, 2 games ago... 11 games ago

In [3]:
#Scale Values
for i in ['Today'] + [str(x) for x in range(2, 12)]:
    usg[i] = usg[i] * 100
usg

,Unnamed: 0,GameID,Player,Today,2,3,4,5,6,7,8,9,10,11
0,0,1997-10-310BOS,walkean02,28.448276,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,1997-10-310BOS,mccarwa01,12.068966,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,1997-10-310BOS,knightr01,10.344828,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,1997-10-310BOS,mercero01,9.482759,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4,1997-10-310BOS,bowenbr01,7.758621,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516590,516590,2019-04-070GSW,templga01,1.769912,11.403509,6.250000,4.504505,5.555556,6.306306,10.091743,2.727273,3.921569,2.608696,6.611570
516591,516591,2019-04-070GSW,wallaty01,8.849558,1.754386,5.357143,0.900901,3.703704,3.603604,4.587156,3.361345,0.892857,1.709402,4.807692
516592,516592,2019-04-070GSW,robinje01,7.079646,5.263158,3.571429,0.900901,3.703704,5.405405,1.834862,1.694915,5.833333,0.917431,1.754386
516593,516593,2019-04-070GSW,motlejo01,7.079646,2.678571,2.777778,0.917431,2.542373,3.333333,2.752294,11.504425,3.539823,1.834862,4.273504


In [4]:
#Reformat GameID and Player Columns to be equivalent between dataframes

def fixID(ID):
    return ID[7:] + ID[4:7]

raw['GameID'] = raw['GameID'].apply(fixID)
raw = raw.rename(columns={'Starters': 'Player'})
raw

,Season,Date,GameID,Team,Location,Player,MP,FG,FGA,3P,...,TRB,AST,STL,BLK,TOV,Non_Shooting,Shooting,Resulting_FT,PTS,Started
0,1,19971031,1997-10-31BOS,CHI,@,jordami01,42.0,7,23,0,...,6,4,2,1,3,2,0,0,30,1
1,1,19971031,1997-10-31BOS,CHI,@,kukocto01,35.8,6,13,0,...,4,2,3,1,3,3,2,1,12,1
2,1,19971031,1997-10-31BOS,CHI,@,harpero01,27.4,0,3,0,...,3,4,4,0,2,1,0,0,1,1
3,1,19971031,1997-10-31BOS,CHI,@,caffeja01,26.9,6,13,0,...,8,2,2,0,2,1,2,2,13,1
4,1,19971031,1997-10-31BOS,CHI,@,longllu01,21.1,4,6,0,...,5,0,0,2,4,8,0,2,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535023,22,20190410,2019-04-10POR,POR,Home,trentga02,48.0,8,19,1,...,2,3,1,1,1,2,0,0,19,1
535024,22,20190410,2019-04-10POR,POR,Home,laymaja01,48.0,7,15,2,...,4,2,1,0,4,2,1,1,19,1
535025,22,20190410,2019-04-10POR,POR,Home,leoname01,42.1,8,13,1,...,11,3,0,0,3,2,2,1,19,1
535026,22,20190410,2019-04-10POR,POR,Home,labissk01,40.6,12,17,2,...,15,1,2,1,2,5,2,3,29,1


In [5]:
usg['GameID'] = usg['GameID'].apply(lambda x: x[:-4] + x[-3:])
del usg['Unnamed: 0']
usg

,GameID,Player,Today,2,3,4,5,6,7,8,9,10,11
0,1997-10-31BOS,walkean02,28.448276,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1997-10-31BOS,mccarwa01,12.068966,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1997-10-31BOS,knightr01,10.344828,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1997-10-31BOS,mercero01,9.482759,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1997-10-31BOS,bowenbr01,7.758621,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
516590,2019-04-07GSW,templga01,1.769912,11.403509,6.250000,4.504505,5.555556,6.306306,10.091743,2.727273,3.921569,2.608696,6.611570
516591,2019-04-07GSW,wallaty01,8.849558,1.754386,5.357143,0.900901,3.703704,3.603604,4.587156,3.361345,0.892857,1.709402,4.807692
516592,2019-04-07GSW,robinje01,7.079646,5.263158,3.571429,0.900901,3.703704,5.405405,1.834862,1.694915,5.833333,0.917431,1.754386
516593,2019-04-07GSW,motlejo01,7.079646,2.678571,2.777778,0.917431,2.542373,3.333333,2.752294,11.504425,3.539823,1.834862,4.273504


In [6]:
#Add the starter variable
started = pd.DataFrame(raw[['GameID', 'Player', 'Started']])

usg = usg.replace(0, np.nan).dropna()
merged = pd.merge(usg, started, on =['GameID', 'Player'], how='inner')



In [7]:
merged

,GameID,Player,Today,2,3,4,5,6,7,8,9,10,11,Started
0,1997-11-18ATL,mutomdi01,15.254237,19.387755,11.818182,19.354839,15.625000,21.568627,15.841584,20.388350,10.526316,14.705882,13.265306,1
1,1997-11-18ATL,corbity01,11.864407,15.306122,11.818182,7.526882,10.416667,10.784314,13.861386,10.679612,4.210526,15.686275,15.306122,1
2,1997-11-18ATL,laettch01,22.881356,11.224490,12.727273,10.752688,17.708333,14.705882,8.910891,17.475728,14.736842,10.784314,10.204082,1
3,1997-11-18ATL,smithst01,10.169492,16.326531,12.727273,22.580645,19.791667,16.666667,23.762376,20.388350,24.210526,16.666667,20.408163,1
4,1997-11-18ATL,blaylmo01,18.644068,15.306122,13.636364,23.655914,10.416667,17.647059,18.811881,18.446602,16.842105,18.627451,15.306122,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497244,2019-04-07GSW,templga01,1.769912,11.403509,6.250000,4.504505,5.555556,6.306306,10.091743,2.727273,3.921569,2.608696,6.611570,0
497245,2019-04-07GSW,wallaty01,8.849558,1.754386,5.357143,0.900901,3.703704,3.603604,4.587156,3.361345,0.892857,1.709402,4.807692,0
497246,2019-04-07GSW,robinje01,7.079646,5.263158,3.571429,0.900901,3.703704,5.405405,1.834862,1.694915,5.833333,0.917431,1.754386,0
497247,2019-04-07GSW,motlejo01,7.079646,2.678571,2.777778,0.917431,2.542373,3.333333,2.752294,11.504425,3.539823,1.834862,4.273504,0


In [8]:
#Add the Age variable
with open('../ages.pickle', 'rb') as pik:
    ages = pickle.load(pik)
    
lst = []
for row in merged.iterrows():
    player = row[1]['Player']
    year = int(row[1]['GameID'][:4])
    age = year - int(ages[player])
    lst.append(age)
    
merged['Age'] = lst
merged

,GameID,Player,Today,2,3,4,5,6,7,8,9,10,11,Started,Age
0,1997-11-18ATL,mutomdi01,15.254237,19.387755,11.818182,19.354839,15.625000,21.568627,15.841584,20.388350,10.526316,14.705882,13.265306,1,31
1,1997-11-18ATL,corbity01,11.864407,15.306122,11.818182,7.526882,10.416667,10.784314,13.861386,10.679612,4.210526,15.686275,15.306122,1,35
2,1997-11-18ATL,laettch01,22.881356,11.224490,12.727273,10.752688,17.708333,14.705882,8.910891,17.475728,14.736842,10.784314,10.204082,1,28
3,1997-11-18ATL,smithst01,10.169492,16.326531,12.727273,22.580645,19.791667,16.666667,23.762376,20.388350,24.210526,16.666667,20.408163,1,28
4,1997-11-18ATL,blaylmo01,18.644068,15.306122,13.636364,23.655914,10.416667,17.647059,18.811881,18.446602,16.842105,18.627451,15.306122,1,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497244,2019-04-07GSW,templga01,1.769912,11.403509,6.250000,4.504505,5.555556,6.306306,10.091743,2.727273,3.921569,2.608696,6.611570,0,33
497245,2019-04-07GSW,wallaty01,8.849558,1.754386,5.357143,0.900901,3.703704,3.603604,4.587156,3.361345,0.892857,1.709402,4.807692,0,25
497246,2019-04-07GSW,robinje01,7.079646,5.263158,3.571429,0.900901,3.703704,5.405405,1.834862,1.694915,5.833333,0.917431,1.754386,0,22
497247,2019-04-07GSW,motlejo01,7.079646,2.678571,2.777778,0.917431,2.542373,3.333333,2.752294,11.504425,3.539823,1.834862,4.273504,0,24


In [9]:
#Perform Regression
x = merged[[str(x) for x in range(2, 12)] + ['Started', 'Age']]
y = merged['Today']
x = sm.add_constant(x)

res = sm.OLS(y, x).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Today   R-squared:                       0.657
Model:                            OLS   Adj. R-squared:                  0.657
Method:                 Least Squares   F-statistic:                 7.930e+04
Date:                Tue, 08 Dec 2020   Prob (F-statistic):               0.00
Time:                        02:48:08   Log-Likelihood:            -1.3769e+06
No. Observations:              497249   AIC:                         2.754e+06
Df Residuals:                  497236   BIC:                         2.754e+06
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3474      0.038     35.596      0.000       1.273       1.422
2              0.1960      0.001    138.681      0.000       0.193       0.199
3              0.1350      0.001     94.018      0.000       0.132       0.138
4              0.0996      0.001     68.882      0.000       0.097       0.102
5              0.0854      0.001     58.879      0.000       0.083       0.088
6              0.0702      0.001     48.322      0.000       0.067       0.073
7              0.0611      0.001     42.060      0.000       0.058       0.064
8              0.0560      0.001     38.650      0.000       0.053       0.059
9              0.0519      0.001     35.952      0.000       0.049       0.055
10             0.0544      0.001     38.087      0.000       0.052       0.057
11             0.0592      0.001     42.306      0.000       0.056       0.062
Started        1.4252      0.013    105.785      0.000       1.399       1.452
Age           -0.0270      0.001    -20.986      0.000      -0.030      -0.024
==============================================================================
Omnibus:                    15576.627   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            24170.561
Skew:                           0.306   Prob(JB):                         0.00
Kurtosis:                       3.889   Cond. No.                         310.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
def split(df):
    limit = int(df.shape[0] / 2)
    
    train = df.iloc[:limit]
    test = df.iloc[limit:]
    
    return train, test

train, test = split(merged)

#Perform Regression
x = train[[str(x) for x in range(2, 12)] + ['Started', 'Age']]
y = train['Today']
x = sm.add_constant(x)

res = sm.OLS(y, x).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Today   R-squared:                       0.667
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                 4.149e+04
Date:                Tue, 08 Dec 2020   Prob (F-statistic):               0.00
Time:                        02:48:08   Log-Likelihood:            -6.9403e+05
No. Observations:              248624   AIC:                         1.388e+06
Df Residuals:                  248611   BIC:                         1.388e+06
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4705      0.055     26.880      0.000       1.363       1.578
2              0.1924      0.002     96.322      0.000       0.189       0.196
3              0.1376      0.002     67.832      0.000       0.134       0.142
4              0.0974      0.002     47.672      0.000       0.093       0.101
5              0.0851      0.002     41.526      0.000       0.081       0.089
6              0.0730      0.002     35.565      0.000       0.069       0.077
7              0.0605      0.002     29.518      0.000       0.057       0.065
8              0.0522      0.002     25.505      0.000       0.048       0.056
9              0.0530      0.002     26.006      0.000       0.049       0.057
10             0.0544      0.002     26.946      0.000       0.050       0.058
11             0.0629      0.002     31.789      0.000       0.059       0.067
Started        1.5149      0.020     76.826      0.000       1.476       1.554
Age           -0.0323      0.002    -17.624      0.000      -0.036      -0.029
==============================================================================
Omnibus:                     6784.085   Durbin-Watson:                   2.039
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10160.825
Skew:                           0.285   Prob(JB):                         0.00
Kurtosis:                       3.810   Cond. No.                         315.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
params = res.params
exes = test.iloc[0][ [str(x) for x in range(2, 12)] +  ['Started', 'Age'] ]


lst= []
for row in test.iterrows():
    exes = row[1][ [str(x) for x in range(2, 12)] +  ['Started', 'Age'] ]
    lst.append(np.dot(params[1:], exes) + params[0])
    
test['Prediction'] = lst

test[['Prediction', 'Today']]

<ipython-input-12-534581efe6bb>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Prediction'] = lst


,Prediction,Today
248624,4.468632,6.504065
248625,5.386655,3.252033
248626,8.679548,14.634146
248627,3.458001,1.626016
248628,10.160290,10.569106
...,...,...
497244,6.348407,1.769912
497245,3.298631,8.849558
497246,3.729752,7.079646
497247,3.464310,7.079646


In [ ]:
x = test['Prediction']
y = test['Today']

x = sm.add_constant(x)

res = sm.OLS(y, x).fit()
res.summary()

In [13]:
usg.shape

(497374, 13)

In [14]:
raw.shape

(535028, 25)